In [1]:
import pennylane as qml
from pennylane import numpy as np
np.random.seed(0)
dev = qml.device('default.qubit', wires=3)

In [2]:
def true(phi, theta, omega):
    qml.Rot(phi, theta, omega, wires=0)

def generator(w):
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)
    
def discriminator(w):
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[1,2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

In [3]:
@qml.qnode(dev)
def true_disc_circuit(phi, theta, omega, disc_weights):
    true(phi, theta, omega)
    discriminator(disc_weights)
    return qml.expval.PauliZ(2)

@qml.qnode(dev)
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval.PauliZ(2)

In [4]:
def prob_real_true(disc_weights):
    true_disc_output = true_disc_circuit(phi, theta, omega, disc_weights)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true

def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true # want to minimize this prob

def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights) 
    return cost

def gen_cost(gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)

In [5]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
eps = 1e-2
gen_weights = np.array([0] + [np.pi] + [0] * 7) + eps * np.random.normal(size=[9])
disc_weights = np.random.normal(size=[9])

opt = qml.GradientDescentOptimizer(0.1)

In [6]:
# train discriminator
for t in range(50):
    disc_weights = opt.step(disc_cost, disc_weights) 
    cost = disc_cost(disc_weights)
    if t % 5 == 0:
        print("Step {}: cost = {}".format(t, cost))

Step 0: cost = -0.1094201780578915
Step 5: cost = -0.3899884226490312
Step 10: cost = -0.6660191175815631
Step 15: cost = -0.8550839212078475
Step 20: cost = -0.9454459581664487
Step 25: cost = -0.9805878247866405
Step 30: cost = -0.993137132834275
Step 35: cost = -0.997489676491659
Step 40: cost = -0.9989863506630714
Step 45: cost = -0.9995000463932013


In [7]:
 # should be close to one at D's optimum
prob_real_true(disc_weights)

0.9998971951842262

In [8]:
 # should be close to zero at D's optimum
prob_fake_true(gen_weights, disc_weights)

0.00024278396180027473

In [9]:
qml.grad(gen_cost, argnum=0)(gen_weights)

array([-8.81608860e-03, -1.99964264e-03, -4.89102336e-03, -1.11998779e-02,
        8.32667268e-17, -1.66533454e-16, -2.77555756e-17, -0.00000000e+00,
        5.55111512e-17])

In [10]:
# train generator
for t in range(200):
    gen_weights = opt.step(gen_cost, gen_weights)
    cost = -gen_cost(gen_weights)
    if t % 5 == 0:
        print("Step {}: cost = {}".format(t, cost))

Step 0: cost = 0.0002664691382996409
Step 5: cost = 0.0004266200858930036
Step 10: cost = 0.0006872486146982104
Step 15: cost = 0.0011111626380133632
Step 20: cost = 0.0018000510248329937
Step 25: cost = 0.0029179304125441785
Step 30: cost = 0.004727717539773968
Step 35: cost = 0.007646628881031847
Step 40: cost = 0.01232586673573638
Step 45: cost = 0.019754518934527177
Step 50: cost = 0.03136834673567118
Step 55: cost = 0.04909734599307797
Step 60: cost = 0.07520378135265438
Step 65: cost = 0.11169015288702155
Step 70: cost = 0.15917286333740976
Step 75: cost = 0.21566031343947284
Step 80: cost = 0.27637357210452695
Step 85: cost = 0.33541691865274725
Step 90: cost = 0.3883501266928646
Step 95: cost = 0.4337177212014884
Step 100: cost = 0.4728490188392854
Step 105: cost = 0.5087778323625882
Step 110: cost = 0.5451977336157728
Step 115: cost = 0.5856632916397977
Step 120: cost = 0.632789783508533
Step 125: cost = 0.687246922110676
Step 130: cost = 0.7468453348018583
Step 135: cost = 0.

In [11]:
# should be close to one at G's optimum
prob_fake_true(gen_weights, disc_weights)

0.9994220324420162

In [12]:
# hasn't changed from D's turn, putting here for comparison
prob_real_true(disc_weights)

0.9998971951842262

In [13]:
 # should be close to zero at joint optimum
disc_cost(disc_weights)

-0.0004751627422099336